6. Реализуйте классификацию документов с помощью стандартных средств из библиотеки `scikit-learn` и простого word embedding'а. Проилюстрируйте пару примеров классификации с помощью [`eli5`](https://github.com/TeamHG-Memex/eli5) 

In [385]:
import numpy as np
import pandas as pd
import sklearn
from gensim.models import Word2Vec
import gensim
import smart_open as sm
from tqdm import tqdm_notebook
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec
from collections import OrderedDict
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [386]:
def read_data(path="data/news.txt.gz"):
    with sm.open(path, encoding='utf-8') as f:
        for line in f:
            cat, headline, text = line.strip().split('\t')
            yield cat, headline, text

def preprocessMessages(text):
    text = text.lower()
    words = word_tokenize(text)
    stop_words = stopwords.words('russian')
    words = [w for w in words if len(w) > 2]
    words = [word for word in words if word not in stop_words]
    words = [morph.parse(word)[0].normal_form for word in words]
    return words


def prepare_files(data):
    texts = []
    labels = []
    for cat, headline, text in tqdm_notebook(data):
        texts.append(preprocessMessages(text))
        labels.append(cat)
    return np.array(texts), labels

In [490]:
# texts, labels = prepare_files(read_data())

In [3]:
def convertLabelsToDigits(labels):
    uniqueLabels = list(OrderedDict.fromkeys(labels))
    dictionary = dict(zip(uniqueLabels, np.arange(10)))
    mappedLabels = []
    for label in labels:
        mappedLabels.append(dictionary[label])
    return mappedLabels

In [317]:
texts = np.load("data/texts.npy", allow_pickle=True)
digitLabels = np.load("data/digitLabels.npy", allow_pickle=True)
labels = np.load("data/labels.npy", allow_pickle=True)

In [6]:
uniqueLabels = list(OrderedDict.fromkeys(labels))
dictionaryLabelsDigits = dict(zip(uniqueLabels, np.arange(10)))
dictionaryDigitsLabels = dict(zip(np.arange(10), uniqueLabels))

In [7]:
digitLabels = convertLabelsToDigits(labels)

In [18]:
# np.save("data/texts", texts)
# np.save("data/digitLabels", digitLabels)
# np.save("data/labels", labels)

In [518]:
def createW2V(texts):
    size = 500
    w2v = Word2Vec(texts, size=size)
    w2v.wv.save_word2vec_format(f'data/w2v_vectors-{size}.bin')
    return w2v

In [ ]:
w2v = gensim.models.KeyedVectors.load_word2vec_format("data/w2v_vectors-500.bin")
# w2v = createW2V(texts)

In [9]:
def convertTextToDigit(text, w2v):
    counter = 0
    summ = 0
    for word in text:
        try:
            summ += sum(w2v.wv[word])
        except:
            pass
        counter += 500
    return summ/counter

In [10]:
digitTexts = [convertTextToDigit(text, w2v) for text in texts]
digitTexts = np.array(digitTexts)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


### Scikit-learn models

In [461]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

In [13]:
xTrain, xTest, yTrain, yTest = train_test_split(digitTexts, digitLabels, test_size=0.20, random_state=42)

In [14]:
model = DecisionTreeClassifier()
model.fit(xTrain.reshape(-1, 1), yTrain)
accuracy_score(yTest, model.predict(xTest.reshape(-1, 1)))

0.1955

In [15]:
rf = RandomForestClassifier(n_estimators=400, n_jobs=-1, max_depth=15, )
rf.fit(xTrain.reshape(-1, 1), yTrain)
accuracy_score(yTest, rf.predict(xTest.reshape(-1, 1)))

0.2225

In [16]:
def rfInfer(text):
    text = np.array(convertTextToDigit(text, w2v))
    print(dictionaryDigitsLabels[rf.predict(text.reshape(-1, 1))[0]])

In [488]:
rfInfer("Шведский хоккеист Матс Сундин назначен советником главного тренера сборной Швеции Пера Мортса.\
         Об этом сообщает официальный сайт Федерации хоккея Швеции.Сундин будет консультировать Мортса на протяжении сезона-2012/13 по спортивным вопросам,\
         а также будет курировать младшие сборные страны. Предполагается, что Сундин также войдет в состав тренерского штаба сборной Швеции на чемпионате мира 2013 года")

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


culture


##### TF-IDF

In [507]:
def read_data(path="data/news.txt.gz"):
    textRaw = []
    with sm.open(path, encoding='utf-8') as f:
        for line in f:
            cat, headline, text = line.strip().split('\t')
            textRaw.append(preprocessMessages(text))
    return textRaw

In [508]:
textRaw = read_data()

In [509]:
def padData(tmp):
    counter = 0
    c = 0
    for text in tmp:
        c += len(text)
        counter += 1 
    avg = int(c/counter)
    for i in range(10000):
        if len(tmp[i]) < 140:
            differ = avg - len(tmp[i])
            tmp[i] = np.concatenate((tmp[i], np.zeros((differ))))
        else:
            tmp[i] = tmp[i][:avg]
    return tmp

In [510]:
tmp = padData(textRaw)

140


In [511]:
def saveCSV(labels, tmp):
    tmp = [' '.join(t) for t in tmp]
    data = pd.DataFrame(columns=['cls', 'data'])
    data["cls"] = labels
    data["data"] = tmp
    data.to_csv("data/data.csv")
    return data

In [512]:
data = saveCSV(digitLabels, tmp)
# data = pd.read_csv("data/data.csv")
data.head()

,cls,data
0,0,парусный гонка giraglia rolex cup пройти среди...
1,1,шведский хоккеист матс сундина назначить совет...
2,2,гран-при конкурс брэнд года/effie получить гор...
3,3,цена американский нефть wti лондонский биржа i...
4,3,"сбербанк выставить продажа долг 21,4 тысяча до..."


In [514]:
xTrain, xTest, yTrain, yTest = train_test_split(data['data'], data['cls'], random_state = 0)
count_vect = CountVectorizer()
xTrain_counts = count_vect.fit_transform(xTrain)
tfidf_transformer = TfidfTransformer()
xTrain_tfidf = tfidf_transformer.fit_transform(xTrain_counts)

In [515]:
mNB = MultinomialNB().fit(xTrain_tfidf, yTrain)
accuracy_score(yTest, mNB.predict(count_vect.transform(xTest)))

0.7584

In [516]:
def mNBInfer(text):
    text = count_vect.transform([text])
    print(dictionaryDigitsLabels[mNB.predict(text)[0]])

In [517]:
mNBInfer("Шведский хоккеист Матс Сундин назначен советником главного тренера сборной Швеции Пера Мортса.\
         Об этом сообщает официальный сайт Федерации хоккея Швеции.Сундин будет консультировать Мортса на протяжении сезона-2012/13 по спортивным вопросам,\
         а также будет курировать младшие сборные страны. Предполагается, что Сундин также войдет в состав тренерского штаба сборной Швеции на чемпионате мира 2013 года")

sport
